In [124]:
import os
from sys import maxsize
import numpy as np
import matplotlib.pyplot as plt

In [39]:
dir_u = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/"
dir_uv = "/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/"
filename_pre = "nash_allocs_random_uv_1"

Us => [1000, 2, 10] <br>
Vs => [1000, 2, 5] <br>
As => [1000, 2, 10] <br>

100 per file * 10 files

In [70]:
def readdat(directory):
    Us = []
    Vs = []
    As = []
    
    for filenum in range(10):
        print(directory + filename_pre + str(filenum))

        with open(directory + filename_pre + str(filenum), 'r') as file:
            for c in range(100):
                line = file.readline().split()
                # print(line)
                u_size = (int(line[0]), int(line[1]))
                utemp = []
                for i in range(u_size[0]):
                    utemp.append([float(x) for x in file.readline().split()])
                Us.append(utemp)
                # print(utemp)

                line = file.readline().split()
                # print(line)
                v_size = (int(line[0]), int(line[1]))
                vtemp = []
                for i in range(v_size[0]):
                    vtemp.append([float(x) for x in file.readline().split()])
                Vs.append(vtemp)
                # print(vtemp)

                line = file.readline().split()
                # print(line)
                alloc_size = (int(line[0]), int(line[1]))
                alloctemp = []
                for i in range(alloc_size[0]):
                    alloctemp.append([int(x) for x in file.readline().split()])
                As.append(alloctemp)
                # print(alloctemp)

                line = file.readline()
                # print(line)
    return (Us, Vs, As)

EF calculations

In [60]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
        
#     print(alloc)
#     print(pairwise_alloc)
#     print(utility)
#     print(p_utility)
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

def envyMap(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
    return envy_map

def envyMap_upto1(allocs, utilities, p_utilities):
    size_agents = np.array(allocs).shape[0]
    size_res = np.array(allocs).shape[1]
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

For additive utilities

In [134]:
Us, Vs, As = readdat(dir_u)
Us = np.array(Us)
Vs = np.array(Vs)
As = np.array(As)
print("Us shape :", Us.shape)
print("Vs shape :", Vs.shape)
print("As shape :", As.shape)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_u/nash_allocs_random_uv_19
Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)


In [135]:
ef_map = []
envymap0 = []
for i in range(1000):
    envymaptemp = envyMap(As[i], Us[i], Vs[i])
    envymap0.append(envymaptemp)
    # print(envymaptemp)
    ef_map.append(not bool(np.sum(envymaptemp<0)))
    
ef_map = np.array(ef_map)
# print(ef_map)
print("EF0 percent : ", np.sum(ef_map)/len(ef_map) * 100)

EF0 percent :  99.9


In [136]:
failtemp = np.argwhere(ef_map==False)[0][0]
print(failtemp)
print(Us[failtemp])
print(Vs[failtemp])
print(As[failtemp])
print(envymap0[failtemp])

626
[[91.94518809 69.44138683 80.53043221 19.7375666  71.29642043 88.42985512
   2.43830985  3.3893151  92.90327996  0.41590553]
 [50.5897397  67.58852187 74.52953722 79.47757075 96.09451101 90.86539601
  70.56537618 42.14652442 97.90497132 92.42195652]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[1 1 1 0 0 1 0 0 1 0]
 [0 0 0 1 1 0 1 1 0 1]]
[[  0.         325.97262469]
 [ -0.77222723   0.        ]]


In [137]:
ef1_map = []
envymap1 = []
for i in range(1000):
    envymaptemp = envyMap_upto1(As[i], Us[i], Vs[i])
    envymap1.append(envymaptemp)
    # print(envymaptemp)
    ef1_map.append(not bool(np.sum(envymaptemp<0)))
    
ef1_map = np.array(ef1_map)
# print(ef_map)
print("EF1 percent : ", np.sum(ef1_map)/len(ef1_map) * 100)


EF1 percent :  100.0


For complementing utilities

In [138]:
Us, Vs, As = readdat(dir_uv)
Us = np.array(Us)
Vs = np.array(Vs)
As = np.array(As)
print("Us shape :", Us.shape)
print("Vs shape :", Vs.shape)
print("As shape :", As.shape)

/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_10
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_11
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_12
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_13
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_14
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_15
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_16
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_17
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_18
/home/shail/work/UIC/ef-alloc-experiments/nash_allocs_random_uv/nash_allocs_random_uv_19
Us shape : (1000, 2, 10)
Vs shape : (1000, 2, 5)
As shape : (1000, 2, 10)


In [139]:
ef_map = []
envymap0 = []
for i in range(1000):
    envymaptemp = envyMap(As[i], Us[i], Vs[i])
    envymap0.append(envymaptemp)
    # print(envymaptemp)
    ef_map.append(not bool(np.sum(envymaptemp<0)))
    
# print(ef_map)
print("EF0 percent : ", np.sum(ef_map)/len(ef_map) * 100)

EF0 percent :  75.7


In [140]:
ef1_map = []
envymap1 = []
for i in range(1000):
    envymaptemp = envyMap_upto1(As[i], Us[i], Vs[i])
    envymap1.append(envymaptemp)
    # print(envymaptemp)
    ef1_map.append(not bool(np.sum(envymaptemp<0)))
    
ef1_map = np.array(ef1_map)
# print(ef_map)
print("EF1 percent : ", np.sum(ef1_map)/len(ef1_map) * 100)


EF1 percent :  99.9


In [141]:
failtemp = np.argwhere(ef1_map==False)[0][0]
print(failtemp)
print(Us[failtemp])
print(Vs[failtemp])
print(As[failtemp])
print(envymap0[failtemp])

227
[[67.45585206 80.77103962 68.16276775 50.75259906 85.8262014  51.96549706
  98.40725085 71.00046533 46.05665978 81.06982838]
 [88.3277973  87.49497943 39.53900983 23.20599537 68.00383777 68.11960718
  88.6035461  49.79262117 89.29883501 99.54084094]]
[[42.45682793 83.04277739 48.97735609 39.80181219 88.17786199]
 [51.55314168 50.47408964 82.43910135 99.55207132  6.26422986]]
[[0 0 1 1 0 0 0 0 1 1]
 [1 1 0 0 1 1 1 1 0 0]]
[[   0.         -169.39980819]
 [ 375.56370264    0.        ]]
